<a href="https://colab.research.google.com/github/nahubn1/Hybrid-Robot-Navigation-System/blob/main/notebooks/model_prototyping/U-Net-FiLM-Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Overview
This notebook demonstrates how to train a U-Net model with FiLM layers for segmentation. It begins with environment setup, verifies the data pipeline, initializes experiment settings, and finally runs the training loop with monitoring and checkpointing.

## Environment Setup
This section mounts Google Drive, clones the project repository, installs required packages, and links the dataset so training can run smoothly.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
COLAB_PROJECT_ROOT = '/content/Thesis_Project'
if not os.path.exists(COLAB_PROJECT_ROOT):
  !git clone https://github.com/nahubn1/Hybrid-Robot-Navigation-System {COLAB_PROJECT_ROOT}
os.chdir(COLAB_PROJECT_ROOT) # Change directory into the project
!git pull # Ensure it's the latest version

Cloning into '/content/Thesis_Project'...
remote: Enumerating objects: 707, done.
remote: Counting objects: 100% (115/115), done.
remote: Compressing objects: 100% (98/98), done.
remote: Total 707 (delta 46), reused 17 (delta 17), pack-reused 592 (from 2)
Receiving objects: 100% (707/707), 420.56 KiB | 1.68 MiB/s, done.
Resolving deltas: 100% (385/385), done.
Already up to date.


In [ ]:
!pip install -r environment/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 122.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 664.8/664.8 MB 141.1 MB/s eta 0:00:01

In [ ]:
import sys
# Add src to path
sys.path.append(os.path.join(os.getcwd(), 'src'))

# Define Drive paths
DRIVE_DATA_PATH = '/content/drive/MyDrive/Thesis_DNN_Planner/data'
DRIVE_MODELS_PATH = '/content/drive/MyDrive/Thesis_DNN_Planner/models'
DRIVE_RESULTS_PATH = '/content/drive/MyDrive/Thesis_DNN_Planner/results'
LOCAL_DATA_PATH = '/content/dataset'

# Copy dataset to the local SSD for faster loading
if not os.path.exists(LOCAL_DATA_PATH):
    !rsync -ah --progress {DRIVE_DATA_PATH}/ {LOCAL_DATA_PATH}/
DATA_PATH = LOCAL_DATA_PATH

# Link Drive storage to local cloned directories
if not os.path.islink('data'):
    !ln -s {DATA_PATH} data
if not os.path.islink('models'):
    !ln -s {DRIVE_MODELS_PATH} models
if not os.path.islink('results'):
    !ln -s {DRIVE_RESULTS_PATH} results

print("✅ Environment setup complete.")

✅ Environment setup complete.


## Phase 2: Full Pipeline Sanity Check
This quick check ensures that the data loader, model, and loss function work together by running a single pass through the pipeline.

In [ ]:
import torch
import numpy as np
from pathlib import Path
from torch.utils.data import DataLoader
from dnn_guidance.data_loader import PathfindingDataset
from dnn_guidance.model import UNetFiLM
from dnn_guidance.loss import DiceFocalLoss

# Create a temporary minimal dataset
_temp_root = Path('tmp_sanity_data')
_samples = _temp_root/'samples'
_gt = _temp_root/'gt'
_samples.mkdir(parents=True, exist_ok=True)
_gt.mkdir(parents=True, exist_ok=True)

grid = np.zeros((200, 200), dtype=np.uint8)
grid[0, 0] = 8  # start
grid[-1, -1] = 9  # goal
np.savez(_samples/'sample0.npz', map=grid, clearance=2.0, step_size=8.0, config=np.array([]))
heatmap = np.zeros((200, 200), dtype=np.float32)
np.savez(_gt/'sample0.npz', heatmap=heatmap)

# Build dataset and dataloader
dataset = PathfindingDataset(_samples, _gt)
loader = DataLoader(dataset, batch_size=1)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = UNetFiLM().to(device)
criterion = DiceFocalLoss()

(grid_batch, robot_batch), target_batch = next(iter(loader))
grid_batch = grid_batch.to(device)
robot_batch = robot_batch.to(device)
target_batch = target_batch.to(device)

logits = model(grid_batch, robot_batch)
print('Logits shape:', logits.shape)
loss = criterion(logits, target_batch)
print('Loss:', loss.item())

assert logits.shape == (1, 1, 200, 200)
assert loss.dim() == 0

loss.backward()
print('✅ Phase 2 Sanity Check Passed: Model, data pipeline, and loss function are fully integrated and functional.')

Logits shape: torch.Size([1, 1, 200, 200])
Loss: 0.6198154091835022
✅ Phase 2 Sanity Check Passed: Model, data pipeline, and loss function are fully integrated and functional.


## Phase 3: Experiment Setup and Initialization
Here we load configuration files, create dataset splits, and initialize the model, optimizer, and learning rate scheduler.

In [ ]:
import yaml
from pathlib import Path

CONFIG_PATH = 'configs/dnn/unet_film_v1_baseline.yaml'
with open(CONFIG_PATH, 'r') as f:
    cfg = yaml.safe_load(f)
print(f'Configuration loaded from {CONFIG_PATH}.')

Configuration loaded from configs/dnn/unet_film_v1_baseline.yaml.


In [ ]:
import random
import numpy as np
import torch

seed = cfg.get('seed', 0)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)
print(f'Random seed set to {seed}.')

Random seed set to 42.


In [ ]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from dnn_guidance.data_loader import PathfindingDataset, _pair_files
from pathlib import Path

DATA_ROOT = Path(DATA_PATH)
samples_dir = DATA_ROOT / cfg['samples_dir']
gt_dir = DATA_ROOT / cfg['ground_truth_dir']
all_pairs = _pair_files(samples_dir, gt_dir)
train_pairs, val_pairs = train_test_split(all_pairs, test_size=cfg['val_split'], random_state=seed)
train_dataset = PathfindingDataset(samples_dir, gt_dir, augment=True)
val_dataset = PathfindingDataset(samples_dir, gt_dir, augment=False)
train_dataset.pairs = train_pairs
val_dataset.pairs = val_pairs
train_loader = DataLoader(train_dataset, batch_size=cfg['batch_size'], shuffle=True, num_workers=cfg['num_workers'])
val_loader = DataLoader(val_dataset, batch_size=cfg['batch_size'], shuffle=False, num_workers=cfg['num_workers'])
print('DataLoaders created.')

ModuleNotFoundError: No module named 'dnn_guidance'

In [ ]:
from dnn_guidance.model import UNetFiLM
from dnn_guidance.config import UNetConfig

model_cfg = UNetConfig.from_yaml('configs/dnn/unet_film.yaml')
model = UNetFiLM(model_cfg)
device = torch.device(cfg['device'])
model = model.to(device)
print(f'Model loaded onto {device}.')

Model loaded onto cuda.


In [ ]:
import torch.optim as optim

opt_name = cfg['optimizer']['name']
optimizer_cls = getattr(optim, opt_name)
optimizer = optimizer_cls(model.parameters(), lr=cfg['optimizer']['lr'], weight_decay=cfg['optimizer']['weight_decay'])
scheduler = None
if cfg['scheduler']['name'] == 'CosineAnnealing':
    t_max = cfg['epochs'] - cfg['scheduler']['warmup_epochs']
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=t_max)
print('Optimizer and Scheduler initialized.')

Optimizer and Scheduler initialized.


In [ ]:
from dnn_guidance.loss import DiceFocalLoss

loss_cfg = cfg['loss']
loss_fn = DiceFocalLoss(dice_weight=loss_cfg['dice_weight'], focal_weight=loss_cfg['focal_weight'], focal_gamma=loss_cfg['focal_gamma'])
print('Loss function initialized.')

Loss function initialized.


## Phase 4: Training Loop with Monitoring and Checkpointing
The training loop iterates over epochs while logging metrics to TensorBoard and saving checkpoints for recovery and evaluation.

In [ ]:
from datetime import datetime
from pathlib import Path
from torch.utils.tensorboard import SummaryWriter
from torch.cuda.amp import GradScaler
from dnn_guidance.trainer import train_one_epoch, validate_one_epoch

# Directories for logging and checkpoints
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
log_root = Path(cfg['log_dir'])
log_dir = log_root / f"{cfg['run_name']}_{timestamp}"
log_dir.mkdir(parents=True, exist_ok=True)
writer = SummaryWriter(log_dir)

ckpt_dir = Path(cfg['checkpoints_dir'])
ckpt_dir.mkdir(parents=True, exist_ok=True)
best_model_path = ckpt_dir / f"{cfg['run_name']}_best_model.pth"
latest_ckpt = ckpt_dir / f"{cfg['run_name']}_latest.pth"

scaler = GradScaler(enabled=cfg.get('use_amp', True))
best_dice = -1.0
epochs_no_improve = 0
start_epoch = 0

# Resume from checkpoint if available
if latest_ckpt.exists():
    ckpt = torch.load(latest_ckpt, map_location=device)
    model.load_state_dict(ckpt['model'])
    optimizer.load_state_dict(ckpt['optimizer'])
    if scheduler and ckpt.get('scheduler'):
        scheduler.load_state_dict(ckpt['scheduler'])
    scaler.load_state_dict(ckpt.get('scaler', {}))
    start_epoch = ckpt.get('epoch', 0) + 1
    best_dice = ckpt.get('best_dice', -1.0)
    epochs_no_improve = ckpt.get('epochs_no_improve', 0)
    print(f"Resumed from epoch {start_epoch}")

/tmp/ipython-input-20-1112439998.py:19: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler(enabled=cfg.get('use_amp', True))


In [15]:
for epoch in range(start_epoch, cfg['epochs']):
    print(f"--- Epoch {epoch+1}/{cfg['epochs']} ---")
    train_loss = train_one_epoch(model, train_loader, optimizer, loss_fn, device, scaler)
    val_loss, val_dice = validate_one_epoch(model, val_loader, loss_fn, device)
    if scheduler:
        scheduler.step()
    lr = optimizer.param_groups[0]['lr']

    writer.add_scalar('loss/train', train_loss, epoch)
    writer.add_scalar('loss/val', val_loss, epoch)
    writer.add_scalar('dice/val', val_dice, epoch)
    writer.add_scalar('lr', lr, epoch)
    print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Dice: {val_dice:.4f} | LR: {lr:.6f}")

    improved = val_dice > best_dice
    if improved:
        best_dice = val_dice
        torch.save(model.state_dict(), best_model_path)
        print("New best model found, saving checkpoint...")
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1
        print(f"No improvement for {epochs_no_improve} epochs...")

    torch.save({
        'epoch': epoch,
        'model': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        'scheduler': scheduler.state_dict() if scheduler else None,
        'scaler': scaler.state_dict(),
        'best_dice': best_dice,
        'epochs_no_improve': epochs_no_improve
    }, latest_ckpt)

    if epochs_no_improve >= cfg['early_stop_patience']:
        print('Early stopping triggered.')
        break

writer.close()

--- Epoch 1/50 ---


Train Loss: 0.3155 | Val Loss: 0.3293 | Val Dice: 0.3635 | LR: 0.000001
No improvement for 1 epochs...
--- Epoch 2/50 ---


Train Loss: 0.3158 | Val Loss: 0.3292 | Val Dice: 0.3634 | LR: 0.000000
No improvement for 2 epochs...
--- Epoch 3/50 ---


Train Loss: 0.3156 | Val Loss: 0.3293 | Val Dice: 0.3633 | LR: 0.000001
No improvement for 3 epochs...
--- Epoch 4/50 ---


Train Loss: 0.3158 | Val Loss: 0.3292 | Val Dice: 0.3636 | LR: 0.000005
No improvement for 4 epochs...
--- Epoch 5/50 ---


...

Train Loss: 0.3158 | Val Loss: 0.3294 | Val Dice: 0.3633 | LR: 0.000019
No improvement for 6 epochs...
--- Epoch 7/50 ---


Train Loss: 0.3157 | Val Loss: 0.3297 | Val Dice: 0.3628 | LR: 0.000030
No improvement for 7 epochs...
--- Epoch 8/50 ---


Train Loss: 0.3158 | Val Loss: 0.3292 | Val Dice: 0.3637 | LR: 0.000043
No improvement for 8 epochs...
--- Epoch 9/50 ---


Train Loss: 0.3157 | Val Loss: 0.3294 | Val Dice: 0.3632 | LR: 0.000059
No improvement for 9 epochs...
--- Epoch 10/50 ---


Train Loss: 0.3160 | Val Loss: 0.3303 | Val Dice: 0.3618 | LR: 0.000076
No improvement for 10 epochs...
Early stopping triggered.
